In [1]:
import argparse
# from dataclasses import dataclass
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.prompts import ChatPromptTemplate
import sys
import os

In [2]:
CHROMA_PATH = "chroma"

PROMPT_TEMPLATE = """
Answer the question based only on the following context:

{context}

---

Answer the question based on the above context: {question}
"""

In [4]:
def main(query_text=None):
    if query_text is None:
        # Create CLI.
        parser = argparse.ArgumentParser()
        parser.add_argument("query_text", type=str, help="The query text.")
        args = parser.parse_args()
        query_text = args.query_text

    # Prepare the DB.
    embedding_function = OpenAIEmbeddings()
    db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_function)

    # Search the DB.
    results = db.similarity_search_with_relevance_scores(query_text, k=3)
    if len(results) == 0 or results[0][1] < 0.7:
        print(f"Unable to find matching results.")
        return

    context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
    prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
    prompt = prompt_template.format(context=context_text, question=query_text)
    print(prompt)

    model = ChatOpenAI()
    response_text = model.invoke(prompt) #model.predict(prompt)

    sources = [doc.metadata.get("source", None) for doc, _score in results]
    formatted_response = f"Response: {response_text}\nSources: {sources}"
    print(formatted_response)

if __name__ == "__main__":
    if "ipykernel_launcher" in sys.argv[0]:
        # Fallback for interactive environment like Jupyter Notebook
        query_text = "What is PyTorch ?"
        main(query_text=query_text)
    else:
        main()

Human: 
Answer the question based only on the following context:

Definition

PyTorch is an open source machine learning and deep learning framework.
- PyTorch allows you to manipulate and process data and write machine learning algorithms using Python code.

Workflow

![[pytorch_workflow.excalidraw.svg]]

Importing PyTorch

Python
import torch
torch.__version__

---

Definition

A high level deep learning [[API]] written in python for neural networks. It supports multiple backend neural network computations and makes implementing neural networks easy.

![[keras.excalidraw.svg]]

---

A s/w library created by google to implement large scale ML models and to solve complex numerical problems

Notebook: https://colab.research.google.com/github/tensorflow/tfx/blob/master/docs/tutorials/tfx/components_keras.ipynb#scrollTo=FmPftrv0lEQy

---

Answer the question based on the above context: What is PyTorch ?

Response: content='PyTorch is an open source machine learning and deep learning frame